In [1]:
# Access parent directories
from sys import path
from os.path import abspath
path += [abspath("../"*i) for i in (1, 2, 3)]

In [2]:
from QLab import Environment, SimpleAgent, QNetwork
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms
from matplotlib import pyplot as plt
import context
from pacman import RandomAgent, Environment, GameController, GameControllerCVTraining
from QLearning import QNetwork, Qptimizer, ReplayMemory, utilities
from tqdm import tqdm
from random import random
import numpy as np
import pygame

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
class CVAgent(SimpleAgent):
    def __call__(self, state):
        state = self.get_state_from_display()
        roll = random()
        if roll > self.epsilon:
            action = utilities.select_action(self.policy_net, state.unsqueeze(0))
            self.perform_action(action)
        else:
            super().__call__(state)
    
    def feedback(self, state, action, reward, new_state):
        state = self.get_state_from_display()
        self.memory.push(state, action, reward, new_state)
        self.optimizer(gamma = self.gamma)
    
    def get_state_from_display(self):
        px_arr = np.array(pygame.surfarray.pixels3d(pygame.display.get_surface()))
        # print(px_arr.shape)
        t = torch.tensor(px_arr, dtype=torch.float)
        # print(t.shape)
        t = t.mean(2)[:, :, None] / 255.
        # print(t.shape)
        t = t.permute(2, 0, 1)
        # print(t.shape)
        t = resize_board(t).float()
        # print(t.shape)
        # raise RuntimeError("break")
        return t

In [17]:
class DeepNetwork(QNetwork):
	"Simple and probably terrible triple layered qnetwork."
	def __init__(self, h, w):
		super().__init__()
		self.conv1 = nn.Conv2d(1, 8, kernel_size=5, stride=2)
		self.conv2 = nn.Conv2d(8, 16, kernel_size=5, stride=2)
		self.connected1 = nn.Linear(59536, 128)
		self.connected2 = nn.Linear(128, 4)

	def forward(self, x):
		x = F.relu(self.conv1(x))
		x = F.relu((self.conv2(x)))
		x = torch.flatten(x, 1)
		x = F.relu(self.connected1(x))
		x = F.relu(self.connected2(x))
		return x

	def train(self, episodes = 1, update_frq = 10, live = False, plot = False, ghosts = Environment.default_ghosts, epsilon_start = 0.9, map = "default.map"):
		Environ = GameControllerCVTraining if live else Environment
		self.epsilon = epsilon_start

		score_history = []
		try:
			for episode in tqdm(range(episodes)):
				env = Environ(self, ghosts, file = f"../../maps/{map}")
				self.epsilon = epsilon_start * (1 - (episode / episodes))

				while not env.terminal():
					next(env)
					if live: env.update()
				score_history.append(env.score)
				if episode % update_frq == 0:
					self.target_net.copy_from(self.policy_net)
				if plot:
					self.optimizer.plot_loss_variance()
					self.optimizer.plot_loss()
					utilities.quickplot(score_history, "Score", "./score")
		except KeyboardInterrupt: ...
		if live:
			env.close()

In [18]:
height, width = Environment().shape
shnet = DeepNetwork(height, width)
agent = CVAgent(shnet)

0it [00:00, ?it/s]


In [19]:
agent.train(episodes=100, update_frq=9, plot=True, live=True, ghosts=Environment.hostile_ghosts)
agent.save()

  0%|          | 0/100 [00:07<?, ?it/s]


RuntimeError: Given groups=1, weight of size [8, 1, 5, 5], expected input[128, 4, 28, 29] to have 1 channels, but got 4 channels instead

: 